In [1]:
import sys
import os
sys.path.insert(0, './../../../DataAndScripts/structured_scripts')

In [2]:
from importlib import reload

import pickle
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import time

import data_analysis as da
import functions_optimal as funopt
import sims_utils as su
import validate_utils as vu

import network as network

# Read Output of Perceptron Fits

In [3]:
data=da.Data_MonkeyMouse('both','./../../../DataAndScripts/experimental_data',calc_delta_r=True)

In [20]:
output_dir = './../../../DataAndScripts/structured_scripts/perceptron_results'

file_idxs = []
for file in os.listdir(output_dir):
    if "Simulation_Model_Fit_Separately_StructModel-seednnls=" in file and\
        "-activation=relu-alpha=1e-05-hidden_layer_sizes=(150, 100, 50)-"+\
            "learning_rate_init=0.002-tol=1e-06-beta_1=0.9-beta_2=0.99" in file and\
        "Simulation_Simulation_" not in file and\
        "Simulation_w_Rates_Simulation" not in file:
        try:
            with open(output_dir+"/"+file, 'rb') as handle:
                output_fit = pickle.load(handle)
                output_fit['best_sims_dict_'+data.this_animals[0]]
                output_fit['full_balance_'+data.this_animals[0]]
            file_idxs.append(file)
        except:
            pass
file_idxs.sort()

In [16]:
maxseedmlp = len(file_idxs)

fit_params = [np.zeros((maxseedmlp,11)),np.zeros((maxseedmlp,11))]
fit_inputs = [np.zeros((maxseedmlp,8)),np.zeros((maxseedmlp,7))]
fit_moms = [np.zeros((maxseedmlp,7,10)),np.zeros((maxseedmlp,6,10))]
fit_preds = [np.zeros((maxseedmlp,7,6)),np.zeros((maxseedmlp,6,6))]
fit_costs = [1e4*np.ones(maxseedmlp),1e4*np.ones(maxseedmlp)]

for seedmlp in range(maxseedmlp):
    with open(output_dir+"/"+file_idxs[seedmlp], 'rb') as handle:
        output_fit = pickle.load(handle)
    for anim_idx in range(2):
        dataset = data.bootstrap_moments[anim_idx]
        fit_params[anim_idx][seedmlp] = [output_fit['best_sims_dict_'+data.this_animals[anim_idx]][k] for k in list(vu.res_param_idxs.keys())[:-2]]
        fit_inputs[anim_idx][seedmlp] = 10**output_fit['best_input_'+data.this_animals[anim_idx]]
        fit_moms[anim_idx][seedmlp] = output_fit['moments_of_r_sim_'+data.this_animals[anim_idx]]
        fit_preds[anim_idx][seedmlp] = output_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]]
        delta_preds = np.zeros_like(dataset[:,:,0])
        delta_preds[:-1,:] = fit_preds[anim_idx][seedmlp].T
        delta_preds[-1,:] = fit_preds[anim_idx][seedmlp].T[1,:] - fit_preds[anim_idx][seedmlp].T[0,:]
        res = (delta_preds-dataset[:,:,0])/dataset[:,:,1]
        res[6,:] *= [0,0][anim_idx]
        if 10**output_fit['best_input_'+data.this_animals[anim_idx]][-2] > 50:
            res[6,:] = 1e8
        params_dict = output_fit['best_sims_dict_'+data.this_animals[anim_idx]]
        if params_dict['GI']*params_dict['gI'] < 4:
            res *= 1e8
        fit_costs[anim_idx][seedmlp] = np.sum(res**2)

In [17]:
fit_idx = np.argsort(fit_costs[0])[0]
print(fit_idx)
print(fit_costs[0][fit_idx])
print(file_idxs[fit_idx])

36
6.979875431006438
Simulation_Model_Fit_Separately_StructModel-seednnls=59-activation=relu-alpha=1e-05-hidden_layer_sizes=(150, 100, 50)-learning_rate_init=0.002-tol=1e-06-beta_1=0.9-beta_2=0.99.pkl


In [18]:
fit_idx = np.argsort(fit_costs[1])[0]
print(fit_idx)
print(fit_costs[1][fit_idx])
print(file_idxs[fit_idx])

57
17.053064611812594
Simulation_Model_Fit_Separately_StructModel-seednnls=78-activation=relu-alpha=1e-05-hidden_layer_sizes=(150, 100, 50)-learning_rate_init=0.002-tol=1e-06-beta_1=0.9-beta_2=0.99.pkl


In [19]:
comb_fit = {}
for anim_idx in range(2):
    fit_idx = np.argsort(fit_costs[anim_idx])[0]
    with open(output_dir+"/"+file_idxs[fit_idx], 'rb') as handle:
        output_fit = pickle.load(handle)
    comb_fit['best_fit_file_'+data.this_animals[anim_idx]] = file_idxs[fit_idx]
    comb_fit['best_input_'+data.this_animals[anim_idx]] = output_fit['best_input_'+\
                                                                     data.this_animals[anim_idx]]
    comb_fit['best_sims_dict_'+data.this_animals[anim_idx]] = output_fit['best_sims_dict_'+\
                                                                         data.this_animals[anim_idx]]
    comb_fit['stim_size_'+data.this_animals[anim_idx]] = output_fit['stim_size_'+data.this_animals[anim_idx]]
    comb_fit['moments_of_r_sim_'+data.this_animals[anim_idx]] =\
        output_fit['moments_of_r_sim_'+data.this_animals[anim_idx]]
    comb_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]] =\
        output_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]]
    comb_fit['balance_'+data.this_animals[anim_idx]] =\
        output_fit['balance_'+data.this_animals[anim_idx]]
    comb_fit['opto_ratio_'+data.this_animals[anim_idx]] =\
        output_fit['opto_ratio_'+data.this_animals[anim_idx]]
    comb_fit['full_balance_'+data.this_animals[anim_idx]] =\
        output_fit['full_balance_'+data.this_animals[anim_idx]]
    comb_fit['full_opto_ratio_'+data.this_animals[anim_idx]] =\
        output_fit['full_opto_ratio_'+data.this_animals[anim_idx]]
    
with open("Simulation_Model_Fit_Separately_StructModel-"+"Best"+'.pkl', 'wb') as handle:
    pickle.dump(comb_fit,handle)
    
comb_fit = {}
for anim_idx in range(2):
    fit_idx = np.argsort(fit_costs[anim_idx])[0]
    with open(output_dir+"/"+file_idxs[fit_idx], 'rb') as handle:
        output_fit = pickle.load(handle)
    comb_fit['best_fit_file_'+data.this_animals[anim_idx]] = file_idxs[fit_idx]
    comb_fit['best_inputs_'+data.this_animals[anim_idx]] = output_fit['best_input_'+\
                                                                     data.this_animals[anim_idx]]
    comb_fit['best_sims_dict_'+data.this_animals[anim_idx]] = output_fit['best_sims_dict_'+\
                                                                         data.this_animals[anim_idx]]
    comb_fit['nc']=data.nc
    
    comb_fit['best_params_'+data.this_animals[anim_idx]] = np.concatenate(([output_fit['best_sims_dict_'+\
                                data.this_animals[anim_idx]][k] for k in list(vu.res_param_idxs.keys())[:-2]],
                                10**output_fit['best_input_'+data.this_animals[anim_idx]][-2:]))
    comb_fit['best_rXs_'+data.this_animals[anim_idx]] = 10**output_fit['best_input_'+data.this_animals[anim_idx]][:-1]
    comb_fit['best_exp_input_'+data.this_animals[anim_idx]] = 10**output_fit['best_input_'+\
                                                                             data.this_animals[anim_idx]]
    comb_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]] =\
        output_fit['predictions_of_r_sim_'+data.this_animals[anim_idx]].T
    comb_fit['balance_'+data.this_animals[anim_idx]] =\
        output_fit['balance_'+data.this_animals[anim_idx]]
    comb_fit['opto_ratio_'+data.this_animals[anim_idx]] =\
        output_fit['opto_ratio_'+data.this_animals[anim_idx]]
    comb_fit['full_balance_'+data.this_animals[anim_idx]] =\
        output_fit['full_balance_'+data.this_animals[anim_idx]]
    comb_fit['full_opto_ratio_'+data.this_animals[anim_idx]] =\
        output_fit['full_opto_ratio_'+data.this_animals[anim_idx]]
    
with open("Model_Fit_Separately_StructModel-"+"Best"+'.pkl', 'wb') as handle:
    pickle.dump(comb_fit,handle)